# Working with Stimuli

So far we have learned how to load and handle spikes. Now we want to look at the stimuli eliciting the spikes to be able to investige the stimulus-response relationship of the recordings.

**You will learn to:**
 - Connect stimuli to spiking responses
 - Analyse multiple trials
 - Plot peristimulus time histogram (PSTH)
 
Let's start again by importing relevant packages.

In [ ]:
# import packages
from matplotlib import pyplot as plt
import numpy as np

%matplotlib inline
plt.rcParams['figure.figsize'] = (15.0, 6.0) # set default size of plots

## 1 - On-Off Stimulus

We will take a look at a stimulus consisting of on-off steps in light intensity. This is roughly how the stimulus looks like.

<img src="onoffsteps.gif" width="200">

The information about the stimulus is stored in a text file. For this stimulus, the times of each step transition is provided in the file.

**Exercise:** Load the stimulus step times from the file `filepath` in to the variable `stimulus`, like we have learned yesterday with the spike trains.

In [ ]:
filepath = ''  # TODO: Provide file

### START CODE HERE ###
# stimulus = np.loadtxt(filepath)
stimulus = np.linspace(0, 14, 21)  # TEMPORARY
### END CODE HERE ###

To understand the loaded data, we will visualize them here with time versus light intensity.

In [ ]:
# Create a list with alternating 0, 0.5 and 1
intensity = np.zeros(stimulus.shape)
intensity[0::2] = 0.5
intensity[1::4] = 1

plt.step(stimulus, intensity, where='post')
plt.xlabel('Time (s)');
plt.ylabel('Intesity');

We now load the spike times.

In [ ]:
# Load the spike timings from file
filepath = ''
spike_times = np.loadtxt(filepath)

## 3 - Alignment

A typical problem when handling data is the alignment of stimulus and recorded data, since it is not easy to start the stimulus and the recording at exactly the same time. Fortunately, we have the timings of both stimulus and spikes, such that we can simply cut off any spikes occuring beyond the first and last stimulus time.

In [ ]:
spike_times = spike_times[spike_times > stimulus[0]]  # Remove spikes before
spike_times = spike_times[spike_times < stimulus[-1]]  # Remove spikes after

In [ ]:
# Bin spike times

# TODO: Is there a function that is introduced before to do this?
# bin_size = ...
# spike_bins = ...

## 2 - Trials

Since we are are dealing with a repeating stimulus, we treat each stimulus cycle as a trial. The responses for each trial can then be stacked to obtain statistics.

To do this we'll have to define the time onset of each trial and reshape the spike times into a matrix with the dimensions `trial length 🞨 number of trials`. The trial (i.e. stimulus) consists of four parts:
 1. Mean grey step
 1. On step
 1. Mean grey step
 1. Off step

**Exercise:** Assuming that all trials were presented at same length, how do we obtain the length of one trial? Store the trial length into the variable `trial_length`. **Hint:** Remeber that indexing starts at zero in python.

In [ ]:
### START CODE HERE ###
trial_length = stimulus[4]
### END CODE HERE ###

Given the `trial_length` and the `bin_size`, we reshape the spikes into trials.

In [ ]:
spike_bins = spike_bins.reshape((-1, trial_length * bin_size))

The binned spikes now have the following dimensions.

In [ ]:
print(spike_bins.shape)

## 3 - Plot Trials

## 4 - Plot PSTH

A peristimulus time histogram (PSTH) ...